In [ ]:
import json
import numpy as np
from ase.io import read as ase_read
from ase.io import write as ase_write
import copy


In [ ]:
with open("oatom_envs_jp_dio-orig_min4.json", "r") as f:
    oatom_envs= json.load(f)

In [ ]:
def filter_bulk_like_envs(all_envs):
    filtered_envs = []
    for env in all_envs:
        grain_fract = env["grain_fract"]
        fract_hcp = env["fract_hcp"]

        if len(grain_fract) == 1 and np.isclose(fract_hcp, 1.0, atol=1e-12):
            filtered_envs.append(env)
    return filtered_envs


In [ ]:
oatom_envs

In [ ]:
bulk_like_envs = filter_bulk_like_envs(oatom_envs)

In [ ]:
len(bulk_like_envs)

In [ ]:
bulk_like_envs[10]

In [ ]:
len(oatom_envs)

In [ ]:
first_20_envs = bulk_like_envs[:20]

In [ ]:
input_xyz = "./jp_dio-orig_min4.xyz"
min_orig_atoms = ase_read(input_xyz)

with open("./noOidx2orig.json", "r") as f:
    noOidx2orig = json.load(f)

# Reverse the mapping: orig index -> noO index
orig2noO = {int(v): int(k) for k, v in noOidx2orig.items()}

grain_ptm_data = np.load("./grains_ptm_111025_min4_fixed.npz")
noO_grains = grain_ptm_data["grains"]
noO_ptm_types = grain_ptm_data["ptm_types"]

xyz_ptm_types = []
for i, atm in enumerate(min_orig_atoms):
    if atm.symbol == "O":
        xyz_ptm_types.append(-1)
    else:
        xyz_ptm_types.append(int(noO_ptm_types[orig2noO[i]]))


In [ ]:
oatom_envs

In [ ]:
def compute_distances_pbc(atoms, central_idx, neighbor_idxs):
    """
    Compute distances between a central atom and its neighbors,
    accounting for periodic boundary conditions.

    Parameters:
    -----------
    atoms : ase.Atoms
        The atomic structure
    central_idx : int
        Index of the central atom
    neighbor_idxs : list of int
        Indices of neighbor atoms

    Returns:
    --------
    dict
        Keys are neighbor indices, values are distances (in Angstroms)
    """
    central_pos = atoms[central_idx].position
    cell = atoms.cell

    distances = {}
    for nidx in neighbor_idxs:
        neighbor_pos = atoms[nidx].position

        # Compute displacement vector
        delta = neighbor_pos - central_pos

        # orthorhombic assertion that really should just be done once
        for i in range(3):
            for j in range(3):
                if i == j:
                    continue
                else:
                    assert np.isclose(cell[i, j], 0.0, atol=1e-12)

        # Apply minimum image convention for PBC
        # For orthorhombic cell: wrap to [-L/2, L/2]
        for i in range(3):
            if atoms.pbc[i]:
                cell_length = cell[i, i]
                delta[i] -= cell_length * np.round(delta[i] / cell_length)

        # Compute distance
        distance = np.linalg.norm(delta)
        distances[nidx] = distance

    return distances

In [ ]:
oatom_envs_dict = {env['index']: env for env in oatom_envs}

In [ ]:
# Function to generate oatom_neighbor_pool with 4 iterations

def generate_oatom_neighbor_pool(central_env, oatom_envs_dict, num_iterations=4):
    """
    Generate a pool of O atom neighbors by iteratively expanding through neighbor shells.

    Parameters:
    -----------
    central_env : dict
        The central O atom environment
    oatom_envs_dict : dict
        Dictionary mapping O atom indices to their environment data
    num_iterations : int
        Number of expansion iterations (default 4)

    Returns:
    --------
    list : Pool of O atom indices including central atom and neighbors
    """
    oatom_neighbor_pool = [central_env['index']]  # Start with central atom
    previous_layer = [central_env['index']]  # Track atoms from previous iteration

    for iteration in range(num_iterations):
        current_layer = []  # New O atoms found in this iteration

        # For each O atom in the previous layer, find its O atom neighbors
        for oatom_idx in previous_layer:
            if oatom_idx not in oatom_envs_dict:
                continue

            env = oatom_envs_dict[oatom_idx]
            neighbor_idxs = env['neighbor_idxs']

            # Find O atoms in the neighbor list
            for nidx in neighbor_idxs:
                # Check if this neighbor is an O atom and not already in pool
                if nidx in oatom_envs_dict and nidx not in oatom_neighbor_pool:
                    oatom_neighbor_pool.append(nidx)
                    current_layer.append(nidx)

        if len(current_layer) == 0:
            break  # No new O atoms found, stop early

        previous_layer = current_layer  # Update for next iteration

    return oatom_neighbor_pool

print("Function defined successfully")

In [ ]:
# Function to find unique Hf neighbors from an O atom neighbor pool

def find_unique_hf_neighbors(oatom_pool, oatom_envs_dict):
    """
    Find all unique Hf atoms in the neighbor lists of O atoms in the pool.

    Parameters:
    -----------
    oatom_pool : list
        List of O atom indices
    oatom_envs_dict : dict
        Dictionary mapping O atom indices to their environment data

    Returns:
    --------
    list : Unique Hf atom indices
    """
    unique_hf_neighbors = set()

    for oatom_idx in oatom_pool:
        if oatom_idx not in oatom_envs_dict:
            print("huh?!?!?!")
            continue

        env = oatom_envs_dict[oatom_idx]
        neighbor_idxs = env['neighbor_idxs']

        # Add all neighbors (which could be Hf or O atoms)
        # We'll filter to only Hf atoms by checking if they're NOT in oatom_envs_dict
        for nidx in neighbor_idxs:
            if nidx not in oatom_envs_dict:  # Not an O atom, so it's Hf
                unique_hf_neighbors.add(nidx)

    return list(unique_hf_neighbors)

print("Function defined successfully")

In [ ]:
# Function to find minimum distance to non-HCP Hf atom

def find_min_distance_to_non_hcp(central_idx, hf_neighbors, atoms, xyz_ptm_types):
    """
    Find the minimum distance from central atom to an Hf neighbor with ptm_type != 2.

    Parameters:
    -----------
    central_idx : int
        Index of the central O atom
    hf_neighbors : list
        List of Hf atom indices
    atoms : ase.Atoms
        The atomic structure
    xyz_ptm_types : list
        PTM types for all atoms in structure

    Returns:
    --------
    float : Minimum distance to non-HCP Hf atom, or 1000000 if none found
    """
    # Compute distances from central atom to all Hf neighbors
    distances = compute_distances_pbc(atoms, central_idx, hf_neighbors)

    # Find minimum distance to Hf atom with ptm_type != 2
    min_distance = 1000000.0
    found_non_hcp = False

    for hf_idx, dist in distances.items():
        ptm_type = xyz_ptm_types[hf_idx]
        if ptm_type != 2:  # Not HCP
            if dist < min_distance:
                min_distance = dist
                found_non_hcp = True

    return min_distance

In [ ]:
results = []

print("Processing first 20 filtered environments...")
print("=" * 80)

for i, env in enumerate(first_20_envs):
    print(f"\nProcessing environment {i+1}/20 (O atom index: {env['index']})")

    # Step 1: Generate O atom neighbor pool
    oatom_pool = generate_oatom_neighbor_pool(env, oatom_envs_dict, num_iterations=4)
    print(f"  O atom neighbor pool size: {len(oatom_pool)}")

    # Step 2: Find unique Hf neighbors from the pool
    hf_neighbors = find_unique_hf_neighbors(oatom_pool, oatom_envs_dict)
    print(f"  Unique Hf neighbors: {len(hf_neighbors)}")

    # Step 3 & 4: Compute distances and find minimum to non-HCP Hf
    min_dist = find_min_distance_to_non_hcp(env['index'], hf_neighbors,
                                             min_orig_atoms, xyz_ptm_types)
    print(f"  Minimum distance to non-HCP Hf: {min_dist:.4f} Å")

    # Store results
    results.append({
        'o_atom_index': env['index'],
        'oatom_pool_size': len(oatom_pool),
        'hf_neighbors_count': len(hf_neighbors),
        'min_distance_to_non_hcp': min_dist
    })

print("\n" + "=" * 80)
print("Processing complete!")
print(f"\nTotal results: {len(results)}")

In [ ]:
results

In [ ]:
results_4K = []

print("Processing first 4000 filtered environments...")
print("=" * 80)

for i, env in enumerate(bulk_like_envs[:4000]):
    print(f"\nProcessing environment {i+1}/4000 (O atom index: {env['index']})")

    # Step 1: Generate O atom neighbor pool
    oatom_pool = generate_oatom_neighbor_pool(env, oatom_envs_dict, num_iterations=4)
    print(f"  O atom neighbor pool size: {len(oatom_pool)}")

    # Step 2: Find unique Hf neighbors from the pool
    hf_neighbors = find_unique_hf_neighbors(oatom_pool, oatom_envs_dict)
    print(f"  Unique Hf neighbors: {len(hf_neighbors)}")

    # Step 3 & 4: Compute distances and find minimum to non-HCP Hf
    min_dist = find_min_distance_to_non_hcp(env['index'], hf_neighbors,
                                             min_orig_atoms, xyz_ptm_types)
    print(f"  Minimum distance to non-HCP Hf: {min_dist:.4f} Å")

    # Store results
    results_4K.append({
        'o_atom_index': env['index'],
        'oatom_pool_size': len(oatom_pool),
        'hf_neighbors_count': len(hf_neighbors),
        'min_distance_to_non_hcp': min_dist
    })

print("\n" + "=" * 80)
print("Processing complete!")
print(f"\nTotal results: {len(results)}")

In [ ]:
results_4K[0]

In [ ]:
with open("sample_4k_bulk-like_env_idxs.json", "w") as f:
    json.dump(results_4K, f, indent=2)

In [ ]:
max([d["min_distance_to_non_hcp"] for d in results_4K])

This is kind of surprising: Situations where you have more than one grain in the environment but also fract_hcp ==1.0

In [ ]:
for env in oatom_envs:
    grain_fract = env["grain_fract"]
    fract_hcp = env["fract_hcp"]
    if np.isclose(fract_hcp,1.0,atol=1e-8) and len(grain_fract) > 1:
        print(env["index"])
        print(grain_fract)


In [ ]:
nnlist_data = np.load("orig_dio_polycrystal_neighborlist10dot4.npz")
nn_i, nn_j, nn_S = nnlist_data["i"] , nnlist_data["j"], nnlist_data["S"]
nn_dict = {}
for k in range(len(nn_i)):
    iidx = int(nn_i[k])
    if iidx not in nn_dict:
        nn_dict[iidx] = []
    nn_dict[iidx].append(int(nn_j[k]))

In [ ]:
xyz_grain_idxs = []
for i,atm in enumerate(min_orig_atoms):
    if atm.symbol == "O":
        xyz_grain_idxs.append(-1)
        continue

    xyz_grain_idxs.append(noO_grains[orig2noO[i]])

In [ ]:
def generate_temp_xyz(index):
    min_orig_out = copy.deepcopy(min_orig_atoms)

    isneighbor = np.zeros(len(min_orig_atoms))
    isneighbor[nn_dict[index]] = 1
    isneighbor[index] = 1
    min_orig_out.set_array("is_neighbor", isneighbor)

    new_symbols = min_orig_out.get_chemical_symbols().copy()
    new_symbols[index] = "Np"
    min_orig_out.set_chemical_symbols(new_symbols)


    min_orig_out.set_array("grain_index", np.array(xyz_grain_idxs))
    min_orig_out.set_array("ptm_type", np.array(xyz_ptm_types))

    ase_write("temp.xyz", min_orig_out, format="extxyz")

In [ ]:
generate_temp_xyz(110329)